# Assignment 5 #
### Due: Friday, November 17th to be submitted via Canvas by 11:59 pm ###
### Total points: **65** ###

- Group 106
- Zihao Zhu
- Yolanda Ojeda

# Q1: Support Vector Machines (10 points)

In this question, we will explore support vector machines for the Spam Base dataset from the UCI repository.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score

In [ ]:
seed = 42

In [ ]:
#!pip install ucimlrepo

In [ ]:
#from ucimlrepo import fetch_ucirepo
# fetch dataset
#spambase = fetch_ucirepo(id=94)

df = pd.read_csv("spambase.data", header=None)
y = df[57]
X = df.drop(columns=[57])

# data (as pandas dataframes)
#X = spambase.data.features
#y = spambase.data.targets
#print("Abstract:", spambase.metadata['abstract'])
#print("Number of instances:", spambase.metadata['num_instances'])
#print("Number of features:", spambase.metadata['num_features'])
#print(spambase.variables['name'])

In [ ]:
X.head()

,0,1,2,3,4,5,6,7,8,9,...,47,48,49,50,51,52,53,54,55,56
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.0,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.0,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.0,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.0,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.0,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191


In [ ]:
seed = 42
y = y.to_numpy().squeeze()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=seed)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

a. (5 points) Implement the following function to train SVMs with a specified kernel type, hyper-parameter search space, and random state on the Spam Base dataset. Do hyper-parameter search over $C$ using [cross_val_score](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html), setting the number of folds to 5. After finding the best C, please use it to train the final model and return both the final model and the best C.

In [ ]:
def search_best_svm(kernel, C_search_space, random_state):
    best_score = -np.inf
    best_C = None

    for C in C_search_space:
        # Initialize an SVM classifier with the specified kernel type, C value, and random state
        model = SVC(kernel=kernel, C=C, random_state=random_state)  # Correct assignment

        # Evaluate accuracy scores using 5-fold cross-validation scores
        scores = cross_val_score(model, X_train, y_train, cv=5)

        # Compute the average score and compare with the current best score to update the best C
        score = np.mean(scores)
        if score > best_score:
            best_score = score
            best_C = C
        print(f"C: {C} Avg Cross Val Score: {np.round(score, 4)}")

    print(f"Best C: {best_C}")

    # Initialize the model using the specified kernel type, best C, and random state;
    # and then fit the model using the training set
    model = SVC(kernel=kernel, C=best_C, random_state=random_state)
    model.fit(X_train, y_train)

    return model, best_C


b. (3 points) Run the function you implemented above to train SVMs with the search space of $C$ being [$0.1, 1, 10, 100$], random state set to 42, with the following three popular kernels: (i) linear (ii) polynomial (iii) RBF (Gaussian). Evaluate your final models on the test set and report their accuracies.

In [ ]:
# Define the C search space and kernels
C_search_space = [0.1, 1, 10, 100]
kernels = ['linear', 'poly', 'rbf']

# Train and evaluate models for each kernel
for kernel in kernels:
    model, best_C = search_best_svm(kernel, C_search_space, random_state=42)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Kernel: {kernel}, Best C: {best_C}, Test Accuracy: {accuracy:.4f}")


C: 0.1 Avg Cross Val Score: 0.9214
C: 1 Avg Cross Val Score: 0.9264
C: 10 Avg Cross Val Score: 0.9283
C: 100 Avg Cross Val Score: 0.9268
Best C: 10
Kernel: linear, Best C: 10, Test Accuracy: 0.9283
C: 0.1 Avg Cross Val Score: 0.6895
C: 1 Avg Cross Val Score: 0.7594
C: 10 Avg Cross Val Score: 0.8428
C: 100 Avg Cross Val Score: 0.8957
Best C: 100
Kernel: poly, Best C: 100, Test Accuracy: 0.9125
C: 0.1 Avg Cross Val Score: 0.896
C: 1 Avg Cross Val Score: 0.9243
C: 10 Avg Cross Val Score: 0.9243
C: 100 Avg Cross Val Score: 0.9123
Best C: 10
Kernel: rbf, Best C: 10, Test Accuracy: 0.9354


c. (2 points) Train a logistic regression model using the training set. Compare its performance with that of the SVMs trained above.

In [ ]:
# Train Logistic Regression model
log_reg = LogisticRegression(random_state=42)
log_reg.fit(X_train, y_train)

# Predict and evaluate the model
y_pred_log_reg = log_reg.predict(X_test)
accuracy_log_reg = accuracy_score(y_test, y_pred_log_reg)
print(f"Logistic Regression Test Accuracy: {accuracy_log_reg:.4f}")

Logistic Regression Test Accuracy: 0.9223


# Question 2 : Ensemble Methods for Classification (25 pts)

In this question, we will compare the performances of different ensemble methods for classification problems: [Bagging](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html), [AdaBoost](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html), [Random Forest](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) and [XGBoost](https://xgboost.readthedocs.io/en/stable/python/python_api.html) classifiers.

We will look at the [GiveMeSomeCredit](https://www.kaggle.com/c/GiveMeSomeCredit) dataset for this question. The dataset is extremely large so for this question we will only consider a subset which has been provided along with the notebook for this assignment. The dataset has already been split into train and test sets.

The task is to predict the probability that someone will experience financial distress in the next two years.

In [ ]:
# Only use this code block if you are using Google Colab.
# If you are using Jupyter Notebook, please ignore this code block. You can directly upload the file to your Jupyter Notebook file systems.
from google.colab import files

## It will prompt you to select a local file. Click on “Choose Files” then select and upload the file.
## Wait for the file to be 100% uploaded. You should see the name of the file once Colab has uploaded it.
uploaded = files.upload()

Saving hw5_data.csv to hw5_data.csv


In [ ]:
import pandas as pd

data = pd.read_csv('hw5_data.csv')
data.drop(data.columns[data.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
data.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,0,0.571373,66,0,0.430620,9274.0,10,0,1,0,0.0
1,0,0.233999,56,0,0.257380,5656.0,12,0,0,0,0.0
2,0,0.299270,33,0,0.114575,4747.0,8,0,0,0,3.0
3,0,0.032165,41,0,0.308326,8490.0,8,0,1,0,0.0
4,0,0.050591,36,0,0.862627,3333.0,8,0,2,0,0.0


In [ ]:
from sklearn.model_selection import train_test_split
y = data['SeriousDlqin2yrs']
X = data.drop(['SeriousDlqin2yrs'],axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 7)

print('train:',X_train.shape, y_train.shape)
print('test:',X_test.shape, y_test.shape)

train: (3750, 10) (3750,)
test: (1250, 10) (1250,)


In [ ]:
import matplotlib.pyplot as plt
from sklearn.model_selection import (train_test_split,GridSearchCV)
from sklearn.metrics import (accuracy_score,roc_auc_score)
from sklearn.ensemble import (RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier)
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from time import time
import xgboost as xgb
%matplotlib inline

In [ ]:
columns_list = list(X.columns)

a. (2.5 pts) Fit a Decision Tree Classifier with random_state = 14 for this classification problem. Report the accuracy_score and roc_auc_score on the test set.

In [ ]:
def fit_classifier(clf):
  # Fit the classifier on the training set
  ### START CODE ###
  clf.fit(X_train, y_train)
  ### END CODE ###
  return clf

In [ ]:
def evaluate_classifier(clf, X_test, y_test):
  # Compute the accuracy_score, and roc_auc_score on the test set
  ### START CODE ###
  y_pred = clf.predict(X_test)
  y_pred_proba = clf.predict_proba(X_test)[:, 1]

  acc_score = accuracy_score(y_test, y_pred)
  auc_score = roc_auc_score(y_test, y_pred_proba)
  ### END CODE ###
  print("Accuracy_score: {}, ROC_AUC_score: {}".format(acc_score, auc_score))

In [ ]:
print("Decision Tree")
# Initialize your decision tree classifier
### START CODE ###
dt_clf = DecisionTreeClassifier(random_state=14)
### END CODE ###

dt_clf = fit_classifier(dt_clf)
evaluate_classifier(dt_clf, X_test, y_test)

Decision Tree
Accuracy_score: 0.888, ROC_AUC_score: 0.5854582176218127


b. (2.5 pts) Create a [Bagging](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html) of 25 classifiers (i.e, n_estimators=25) with random_state=14. Please use Decision Tree Classifier with random_state=14 as the base classifier. Report accuracy_score and roc_auc_score on the test data for this emsemble classifier.

In [ ]:
print("Bagging of Decesion Trees")
# Initialize your bagging classifier
### START CODE ###
bag_clf = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(random_state=14),
    n_estimators=25,
    random_state=14
)
### END CODE ###

bag_clf = fit_classifier(bag_clf)
evaluate_classifier(bag_clf, X_test, y_test)

Bagging of Decesion Trees


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Accuracy_score: 0.9256, ROC_AUC_score: 0.7857106791214341


c. (5 pts) In this question, you will fit a [Random Forest](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) model on the training data for this classification task.

1. First, please find the best parameters (including *n_estimators*, *max_features* and *criterion*) using [GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Report the optimal parameters obtained by GridSearch.
2. Fit a model using the best parameters, and report the [confusion matrix](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html) and [roc_auc_score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html#sklearn.metrics.roc_auc_score) on test data.

In [ ]:
def grid_search_for_classifier(clf, param_grid, X_train, y_train):
  # Grid search
  grid_search = GridSearchCV(clf, param_grid=param_grid)

  # Conduct grid search using the training set (1 line of code only)
  ### START CODE ###
  grid_search.fit(X_train, y_train)
  ### END CODE ###
  print(grid_search.best_params_)

  # Set the best paramters for your clf (1 line of code only)
  ### START CODE ###
  clf.set_params(**grid_search.best_params_)
  ### END CODE ###
  return clf

In [ ]:
def train_and_evaluate_classifier(clf, X_train, y_train, X_test, y_test):
  t0 = time()
  # Fit your classifier on the training set
  ### START CODE ###
  clf.fit(X_train, y_train)
  ### END CODE ###
  print("training time", round(time()-t0, 3), "s")

  t0 = time()
  y_pred = clf.predict(X_test)
  print("predict time", round(time()-t0, 3), "s")

  print("Confusion matrix: ")
  # Print the confusion matrix computed from the test set (1 line of code only)
  ### START CODE ###
  print(confusion_matrix(y_test, y_pred))
  ### END CODE ###


  ### START CODE ###
  y_pred_proba = clf.predict_proba(X_test)[:, 1]
  acc_score = accuracy_score(y_test, y_pred)
  auc_score = roc_auc_score(y_test, y_pred_proba)
  ### END CODE ###

  print("Accuracy: {}, AUC_ROC: {}".format(acc_score, auc_score))
  return clf

In [ ]:
param_grid = {"n_estimators": [1, 10, 50, 100],
              "max_features": [1, 5, 10, "auto"],
              "criterion": ['gini','entropy'],
              "random_state": [17]}

# Initialize your random forest classifier
### START CODE ###
rf_clf = RandomForestClassifier(random_state=14)
### END CODE ###
rf_clf = grid_search_for_classifier(rf_clf, param_grid, X_train, y_train)
train_and_evaluate_classifier(rf_clf, X_train, y_train, X_test, y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomFor

{'criterion': 'entropy', 'max_features': 1, 'n_estimators': 100, 'random_state': 17}
training time 0.404 s
predict time 0.026 s
Confusion matrix: 
[[1162    3]
 [  82    3]]
Accuracy: 0.932, AUC_ROC: 0.8375612219136582


RandomForestClassifier(criterion='entropy', max_features=1, random_state=17)

d. (10 pts) This time, let us use [AdaBoost](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html#sklearn.ensemble.AdaBoostClassifier) and [XGBoost](https://xgboost.readthedocs.io/en/stable/python/python_api.html) for the same task. For AdaBoost and XGBoost, please respectively find the best parameters (including *n_estimators, learning_rate*); fit your model using the best parameters, and report the confusion matrix and roc_auc_score on test data.

In [ ]:
param_grid = {"n_estimators": [10, 100],
          "learning_rate": [0.01, 0.1, 0.5],
          "random_state": [17]
          }

In [ ]:
# Initialize your AdaBoost classifier
### START CODE ###
ab_clf = AdaBoostClassifier(random_state=17)
### END CODE ###
ab_clf = grid_search_for_classifier(ab_clf, param_grid, X_train, y_train)
train_and_evaluate_classifier(ab_clf, X_train, y_train, X_test, y_test)

{'learning_rate': 0.1, 'n_estimators': 100, 'random_state': 17}
training time 0.474 s
predict time 0.032 s
Confusion matrix: 
[[1153   12]
 [  72   13]]
Accuracy: 0.9328, AUC_ROC: 0.8390254986114618


AdaBoostClassifier(learning_rate=0.1, n_estimators=100, random_state=17)

In [ ]:
# Initialize your XGBoost classifier
### START CODE ###
xgb_clf = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
### END CODE ###
xgb_clf = grid_search_for_classifier(xgb_clf, param_grid, X_train, y_train)
train_and_evaluate_classifier(xgb_clf, X_train, y_train, X_test, y_test)

{'learning_rate': 0.01, 'n_estimators': 100, 'random_state': 17}
training time 0.133 s
predict time 0.006 s
Confusion matrix: 
[[1163    2]
 [  84    1]]
Accuracy: 0.9312, AUC_ROC: 0.8267962635698057


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.01, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None, random_state=17, ...)

f. (5 pts) Compare the performance of decision tree from part a) with the ensemble methods. Briefly explain which of the three ensemble methods performed better and why?

**Answer**:


As the previous result shows, the Decision Tree achieved an accuracy of 88.8% and an ROC-AUC score of 58.5%. When compared to ensemble methods, the Bagging of Decision Trees showed a substantial improvement with an accuracy of 92.56% and an ROC-AUC score of 78.57%. The Bagging ensemble method combines multiple decision trees to improve overall performance. The Random Forest, a variant of bagging, further enhanced the results with an accuracy of 93.28% and an ROC-AUC score of 83.90%.  Higher accuracy indicates a better overall performance of the model. AUC is a measure of a model's ability to distinguish between classes. A higher ROC AUC score suggests that the model is better at classifying positive and negative cases. The confusion matrix also shows the number of true positive, true negative, false positive, and false negative predictions. which the performance in terms of different types of errors in the three ensemble methods are realtively good in classifying.

The superior performance of ensemble methods, especially Bagging and Random Forest, can be attributed to their ability to mitigate overfitting by aggregating the predictions of multiple base learners. By combining diverse decision trees, these methods capture a broader range of patterns in the data, leading to improved generalization on unseen instances. The ensemble methods excel in handling complex relationships within the dataset, resulting in better predictive performance compared to a standalone Decision Tree. Among the ensemble methods, Random Forest demonstrated the highest accuracy and ROC-AUC score, indicating its effectiveness in this classification task.

# Q3: CatBoost (10 points)

In this question you will learn about a boosting algorithm known as **CatBoost**. Please go through the two videos specified below to get a better understanding of the CatBoost algorithm and answer the questions that follow.

[Part-1](https://www.youtube.com/watch?v=KXOTSkPL2X4&ab_channel=StatQuestwithJoshStarmer)
[Part - 2](https://www.youtube.com/watch?v=3Bg2XRFOTzg&t=242s&ab_channel=StatQuestwithJoshStarmer)



a. **(5 points)** Briefly explain Ordered Target Encoding. What challenge does it try to address?

b. **(5 points)** Briefly describe the main advantages and disadvantages of CatBoost as compared to XGBoost.

**Answer**:
a) Ordered Target Encoding is a technique used in feature engineering for machine learning, particularly for handling categorical variables. It addresses a common challenge in dealing with categorical data: how to effectively encode these categories into numerical values that can be used in machine learning algorithms, especially when there are many categories or when the categories have an inherent order that might be significant for the prediction task.

The primary challenge that Ordered Target Encoding tries to address is the efficient and meaningful encoding of categorical variables, especially when:

High Cardinality: Traditional encoding methods like One-Hot Encoding can lead to a huge increase in dataset dimensions (many new columns) when dealing with categorical variables having many categories (high cardinality). This can increase computational complexity and may lead to overfitting.

Inherent Order: When the categorical variable has an inherent order that is relevant to the target variable, Ordered Target Encoding can capture this relationship more effectively than methods that treat each category as completely independent.

Overfitting with Simple Target Encoding: While simple target encoding can capture the relationship between categorical features and the target variable, it can easily overfit the training data, especially with small datasets or categories with few observations. Ordered Target Encoding addresses this by incorporating methods to reduce overfitting.


b) CatBoost and XGBoost are both powerful gradient boosting libraries with distinct advantages and disadvantages. CatBoost stands out for its efficient handling of categorical features, eliminating the need for one-hot encoding and reducing preprocessing complexities. It also offers built-in cross-validation and robustness to overfitting, thanks to internal regularization techniques. However, CatBoost may have a slightly longer training time, especially on smaller datasets, and its community and documentation might not be as extensive as XGBoost.

On the other hand, XGBoost boasts widespread adoption, a large and active community, and flexibility in regularization parameters, making it a reliable and well-supported choice. It excels in speed, particularly on smaller to medium-sized datasets, and provides efficient GPU support, though not as optimized as CatBoost. However, XGBoost requires explicit handling of categorical features through one-hot encoding, leading to increased memory usage and potentially longer preprocessing times. Choosing between CatBoost and XGBoost depends on the specific needs of the dataset, with CatBoost being advantageous for categorical feature-rich data, while XGBoost might be preferable for its speed and well-established community when dealing with other types of datasets.

# Q4: Convolutional Neural Network (20 points)
In this question, we will continue our exercise on the SVHN classification task from the previous homework, but this time we will be using Convolutional Neural Networks.

In [ ]:
import numpy as np
import random
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(seed)
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)

In [ ]:
transform=torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0.4376821, 0.4437697, 0.47280442), (0.19803012, 0.20101562, 0.19703614))
        ])

train_dataset = torchvision.datasets.SVHN(root='.', split='train', transform=transform, download=True)
test_dataset = torchvision.datasets.SVHN(root='.', split='test', transform=transform, download=True)

100%|██████████| 182040794/182040794 [00:11<00:00, 16384108.73it/s]


100%|██████████| 64275384/64275384 [00:02<00:00, 26013566.15it/s]


In [ ]:
train_num = int(len(train_dataset) * 0.8)
val_num = len(train_dataset) - train_num
# Randomly split the training dataset into training dataset and validation dataset
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_num, val_num])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=256, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)

a. (10 points) Build a convolutional neural network with the following sequential configuration. If not specified, please use the default setting of torch.nn.Conv2d. The output of the convolution layers will be fed into a fully-connected MLP. Then train the model with Adam optimizer (lr=1e-3) for 10 epochs. You should be able to achieve test accuracy of over 85%.



> Layer 1
*   2d convolution (# input channel=3, # output channel=16, kernel size=3, padding=1)
*   2d batch normalization
*   Relu activation

> Pool 1
*   2d max pooling (kernel size=2)

> Layer 2
*   2d convolution (# output channel=16, kernel size=3, padding=1)
*   2d batch normalization
*   Relu activation

> Pool 2
*   2d max pooling (kernel size=2)

> Layer 3
*   2d convolution (# output channel=32, kernel size=3, padding=1)
*   2d batch normalization
*   Relu activation

> Pool 3
*   2d max pooling (kernel size=2)

References:

*   https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html
*   https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html
*   https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html
*   https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html






In [ ]:
class CNN(nn.Module):
    def __init__(self, pool=True):
        super(CNN, self).__init__()
        self.pool = pool

        # Create convolutional layers
        ### START CODE ###
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU()
        )

        self.pool1 = nn.MaxPool2d(kernel_size=2)

        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 16, kernel_size=3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU()
        )

        self.pool2 = nn.MaxPool2d(kernel_size=2)

        self.layer3 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU()
        )
        self.pool3 = nn.MaxPool2d(kernel_size=2)
        ### END CODE ###

        # Create fully connected layers (nn.Linear)
        if self.pool:
            self.mlp1 = nn.Linear(32*4*4, 50)
        else:
            self.mlp1 = nn.Linear(32*32*32, 50)

        self.mlp2 = nn.Linear(50, 50)
        self.mlp3 = nn.Linear(50, 10)

    def forward(self, x):
        x = self.layer1(x)
        if self.pool:
            x = self.pool1(x)

        x = self.layer2(x)
        if self.pool:
            x = self.pool2(x)

        x = self.layer3(x)
        if self.pool:
            x = self.pool3(x)
        x = x.reshape(x.shape[0], -1)

        x = F.relu(self.mlp1(x))
        x = F.relu(self.mlp2(x))
        x = self.mlp3(x)

        return x

In [ ]:
def train(model, loader, optimizer):
    model.train()
    total_loss = 0
    total_num = 0
    for data, target in tqdm(loader):
        out = model(data)
        # Calculate loss based on model output and target
        loss = F.nll_loss(F.log_softmax(out, dim=1), target)

        # Use the optimizer to perform backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        batch_size = len(target)
        total_loss += loss.item() * batch_size
        total_num += batch_size
    avg_loss = total_loss / total_num
    return avg_loss

@torch.no_grad()
def eval(model, loader):
    model.eval()
    total_loss = 0
    total_correct = 0
    total_num = 0
    for data, target in tqdm(loader):
        out = model(data)
        # Calculate loss based on model output and target
        loss = F.nll_loss(F.log_softmax(out, dim=1), target)

        # Get model's prediction
        pred = torch.argmax(out, dim=1)

        # Count number of correct predictions
        correct = accuracy_score(target, pred, normalize=False)

        total_correct += correct
        batch_size = len(target)
        total_loss += loss.item() * batch_size
        total_num += batch_size
    avg_loss = total_loss / total_num
    acc = total_correct / total_num
    return avg_loss, acc

In [ ]:
model1 = CNN(pool=True)
optimizer = torch.optim.Adam(model1.parameters(), lr=1e-3)
best_acc = -np.inf
epochs = 10
for e in range(1, epochs + 1):
    train_loss = train(model1, train_loader, optimizer)
    val_loss, val_acc = eval(model1, val_loader)
    if val_acc > best_acc:
        best_acc = val_acc
        best_model1 = model1
    print(f"Epoch: {e} Train Loss: {train_loss} Val Loss: {val_loss} Val Acc: {val_acc}")

100%|██████████| 58/58 [00:08<00:00,  6.49it/s]


Epoch: 1 Train Loss: 1.22839825788198 Val Loss: 0.647812260749592 Val Acc: 0.8073983073983074


100%|██████████| 58/58 [00:10<00:00,  5.74it/s]


Epoch: 2 Train Loss: 0.5592806533471013 Val Loss: 0.5003044925001583 Val Acc: 0.8536718536718537


100%|██████████| 58/58 [00:09<00:00,  5.99it/s]


...

Epoch: 8 Train Loss: 0.31703089203619733 Val Loss: 0.3720417320687622 Val Acc: 0.8904586404586404


100%|██████████| 58/58 [00:09<00:00,  5.97it/s]


Epoch: 9 Train Loss: 0.3009738132185969 Val Loss: 0.3756331554648331 Val Acc: 0.8910728910728911


100%|██████████| 58/58 [00:09<00:00,  5.88it/s]

Epoch: 10 Train Loss: 0.29225769216071906 Val Loss: 0.37330560991502415 Val Acc: 0.891960141960142


In [ ]:
_, test_acc = eval(best_model1, test_loader)
print(f"Test accuracy: {np.round(test_acc, 3)}")

100%|██████████| 102/102 [00:17<00:00,  5.97it/s]

Test accuracy: 0.889


b. (5 points) Use torch-summary to print a summary of the model. The number of parameters should be less than the one of the MLP we trained in the previous homework. Why does it have less number of parameters but have higher accuracy?

Reference
*   https://pypi.org/project/torch-summary/


In [ ]:
!pip install torch-summary

In [ ]:
from torchsummary import summary


summary(best_model1, (3, 32, 32))

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 16, 32, 32]          --
|    └─Conv2d: 2-1                       [-1, 16, 32, 32]          448
|    └─BatchNorm2d: 2-2                  [-1, 16, 32, 32]          32
|    └─ReLU: 2-3                         [-1, 16, 32, 32]          --
├─MaxPool2d: 1-2                         [-1, 16, 16, 16]          --
├─Sequential: 1-3                        [-1, 16, 16, 16]          --
|    └─Conv2d: 2-4                       [-1, 16, 16, 16]          2,320
|    └─BatchNorm2d: 2-5                  [-1, 16, 16, 16]          32
|    └─ReLU: 2-6                         [-1, 16, 16, 16]          --
├─MaxPool2d: 1-4                         [-1, 16, 8, 8]            --
├─Sequential: 1-5                        [-1, 32, 8, 8]            --
|    └─Conv2d: 2-7                       [-1, 32, 8, 8]            4,640
|    └─BatchNorm2d: 2-8                  [-1, 32, 8, 8]            64
|    └─R

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 16, 32, 32]          --
|    └─Conv2d: 2-1                       [-1, 16, 32, 32]          448
|    └─BatchNorm2d: 2-2                  [-1, 16, 32, 32]          32
|    └─ReLU: 2-3                         [-1, 16, 32, 32]          --
├─MaxPool2d: 1-2                         [-1, 16, 16, 16]          --
├─Sequential: 1-3                        [-1, 16, 16, 16]          --
|    └─Conv2d: 2-4                       [-1, 16, 16, 16]          2,320
|    └─BatchNorm2d: 2-5                  [-1, 16, 16, 16]          32
|    └─ReLU: 2-6                         [-1, 16, 16, 16]          --
├─MaxPool2d: 1-4                         [-1, 16, 8, 8]            --
├─Sequential: 1-5                        [-1, 32, 8, 8]            --
|    └─Conv2d: 2-7                       [-1, 32, 8, 8]            4,640
|    └─BatchNorm2d: 2-8                  [-1, 32, 8, 8]            64
|    └─R

To understand why the Convolutional Neural Network (CNN) has fewer parameters but achieves higher accuracy compared to the Multi-Layer Perceptron (MLP) from the previous homewori can consider the following factors:

Convolutional Layers, CNNs utilize convolutional layers that are specifically designed for processing grid-like data, such as images. These layers use a small set of learnable filters (kernels) that slide across the input data to capture spatial patterns. Unlike fully connected layers in MLPs, convolutional layers share weights across different spatial locations, significantly reducing the number of parameters. This weight sharing allows CNNS to etticiently capture local patterns and edges in the input.

Pooling Layers: CNN architectures often include pooling layers (e.g., max-pooling) that down-sample the spatial dimensions of feature maps. Pooling layers reduce the number of parameters and comouratong complex wnie retaining importancreatures.nis cown-samonne melos in caournethe moscreevancinormation anomakes the nework more ronust to sma variations in Input.

Hierarchical Feature Extraction: CNNs are designed to capture hierarchical features in data. The early layers of a CNN detect simple features like edges and corners, while subsequent layers combine these low-level features to recognize more complex patterns and shapes. This hierarchical feature extraction enables CNNs to represent data in a more compact and
Informative manner, requiring felter parameters to achieve high accuracy.

Translation Invariance: CNN architectures inherently possess translation invariance, which means they can recognize patterns regardless of their position within the input. This property is particularly advantageous for tasks like image classification, where the same object can appear in different locations within an image. By reusing weights across the input, CNNs can effectively learn translation-invariant features.

In summary, CNNs are tailored for tasks involving grid-like data and leverage weight sharing, pooling, and hierarchical feature extraction to efficiently capture essential information while keeping the number of parameters low. This efficiency in feature representation and the network's ability to recognize patterns at different levels of abstraction contribute to its higher accuracy despite having fewer parameters compared to MPs

c. (5 points) Train another CNN with the pool option set to False. What are the differences in terms of accuracy or computation caused by disabling max pooling? What are the effects of pooling operations in CNNs? (This might take some time. Watch a TV show while you're waiting for the results..)

In [ ]:
model2 = CNN(pool=False)
optimizer = torch.optim.Adam(model2.parameters(), lr=1e-3)
best_acc = -np.inf
epochs = 10
for e in range(1, epochs + 1):
    train_loss = train(model2, train_loader, optimizer)
    val_loss, val_acc = eval(model2, val_loader)
    if val_acc > best_acc:
        best_acc = val_acc
        best_model2 = model2
    print(f"Epoch: {e} Train Loss: {train_loss} Val Loss: {val_loss} Val Acc: {val_acc}")

100%|██████████| 58/58 [00:17<00:00,  3.32it/s]


Epoch: 1 Train Loss: 1.0621510849543352 Val Loss: 0.5923077000023854 Val Acc: 0.826986076986077


100%|██████████| 58/58 [00:13<00:00,  4.39it/s]


Epoch: 2 Train Loss: 0.5078498294635007 Val Loss: 0.4920821153105461 Val Acc: 0.8549003549003549


100%|██████████| 58/58 [00:13<00:00,  4.18it/s]


...

Epoch: 8 Train Loss: 0.21606723618897936 Val Loss: 0.45245430674866643 Val Acc: 0.8746246246246246


100%|██████████| 58/58 [00:13<00:00,  4.15it/s]


Epoch: 9 Train Loss: 0.1930729129533395 Val Loss: 0.4546862811972917 Val Acc: 0.878992628992629


100%|██████████| 58/58 [00:16<00:00,  3.49it/s]

Epoch: 10 Train Loss: 0.17116291668800332 Val Loss: 0.4716499082026116 Val Acc: 0.8777641277641277


In [ ]:
_, test_acc = eval(best_model2, test_loader)
print(test_acc)

100%|██████████| 102/102 [00:29<00:00,  3.49it/s]

0.8574830977258758


In [ ]:
summary(best_model2, (3, 32, 32))

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 16, 32, 32]          --
|    └─Conv2d: 2-1                       [-1, 16, 32, 32]          448
|    └─BatchNorm2d: 2-2                  [-1, 16, 32, 32]          32
|    └─ReLU: 2-3                         [-1, 16, 32, 32]          --
├─Sequential: 1-2                        [-1, 16, 32, 32]          --
|    └─Conv2d: 2-4                       [-1, 16, 32, 32]          2,320
|    └─BatchNorm2d: 2-5                  [-1, 16, 32, 32]          32
|    └─ReLU: 2-6                         [-1, 16, 32, 32]          --
├─Sequential: 1-3                        [-1, 32, 32, 32]          --
|    └─Conv2d: 2-7                       [-1, 32, 32, 32]          4,640
|    └─BatchNorm2d: 2-8                  [-1, 32, 32, 32]          64
|    └─ReLU: 2-9                         [-1, 32, 32, 32]          --
├─Linear: 1-4                            [-1, 50]                  1,638,450
├

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 16, 32, 32]          --
|    └─Conv2d: 2-1                       [-1, 16, 32, 32]          448
|    └─BatchNorm2d: 2-2                  [-1, 16, 32, 32]          32
|    └─ReLU: 2-3                         [-1, 16, 32, 32]          --
├─Sequential: 1-2                        [-1, 16, 32, 32]          --
|    └─Conv2d: 2-4                       [-1, 16, 32, 32]          2,320
|    └─BatchNorm2d: 2-5                  [-1, 16, 32, 32]          32
|    └─ReLU: 2-6                         [-1, 16, 32, 32]          --
├─Sequential: 1-3                        [-1, 32, 32, 32]          --
|    └─Conv2d: 2-7                       [-1, 32, 32, 32]          4,640
|    └─BatchNorm2d: 2-8                  [-1, 32, 32, 32]          64
|    └─ReLU: 2-9                         [-1, 32, 32, 32]          --
├─Linear: 1-4                            [-1, 50]                  1,638,450
├

Disabling max pooling in a CNN has dual consequences. Firstly, the spatial resolution of feature maps remains higher, preserving fine-grained spatial details but potentially making the model more sensitive to small translations. Secondly, it leads to an increase in the number of parameters in the subsequent fully connected layers, possibly raising the risk of overfitting. The effects of pooling operations in CNNs are multifold: pooling facilitates downsampling, contributing to translation invariance and noise reduction. It also enhances the receptive field of neurons, allowing them to capture information from a larger portion of the input. The decision to use or omit max pooling involves a trade-off between maintaining spatial information and managing computation, often necessitating experimentation to assess the impact on accuracy and efficiency.